## This notebook focuses on different ways to embed text as vectors using the Keras library

In essence, the Embedding layer is like a dictionary that maps integer indices to dense vectors:

Word index -> Embedding layer -> Corresponding word vector

In [7]:
from keras.layers import Embedding
from keras.datasets import imdb
from keras import preprocessing

In [ ]:
# Ensure MacOS ARM chip GPU is being used


In [4]:
# 1000 - the number of possible tokens
# 64 - the dimensionality of the encodings
embedding_layer = Embedding(1000, 64)

In [8]:
# Loading the data
# Only loads the 10,000 most common words
max_features = 10000
maxlen = 20
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17464789/17464789 [==============================] - 7s 0us/step
